<a href="https://colab.research.google.com/github/fleckc/visSP23/blob/main/ps2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

Looking at NJ voting habits and where voters live/work, using election results from 2000, the 2000 census population counts, and the 2000 census migration patterns in order to retain year similarity.

In [2]:
njvote=pd.read_csv("https://raw.githubusercontent.com/fleckc/visSP23/main/NJ2000ballots.csv")

In [7]:
njpop=pd.read_csv("https://raw.githubusercontent.com/fleckc/visSP23/main/njpop2000.csv")

In [76]:
njmigrate=pd.read_fwf("https://github.com/fleckc/visSP23/raw/main/countymigrationinflow.txt")

In [ ]:
njvote=njvote.rename(columns={'COUNTY': 'County'})
njvote2=njvote.replace('County', njpop)
njvote2

In [ ]:
njpop

In [ ]:
njpop=njpop.drop(njpop.index[21:31])
njpop

In [ ]:
njmigrate

In [77]:
njmigrate=njmigrate.rename(columns={'01001': 'StateCounty2000',
                 '01003': 'StateCounty1995',
                 '87': 'InflowMigration'})

In [83]:
njmigrate.dtypes

StateCounty2000    int64
StateCounty1995    int64
InflowMigration    int64
dtype: object

In [90]:
njmigratenjonly=njmigrate.query('StateCounty2000 > 34000 & StateCounty2000 < 35000')
njmigratenjonly.head()

,StateCounty2000,StateCounty1995,InflowMigration
408749,34001,1123,27
408750,34001,2020,8
408751,34001,2090,13
408752,34001,2122,4
408753,34001,2240,24
